<a href="https://colab.research.google.com/github/neuroidss/silent_speech/blob/main/silent_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content

/content


In [3]:
!git clone https://github.com/dgaddy/silent_speech.git

Cloning into 'silent_speech'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 17), reused 33 (delta 9), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [5]:
#mkdir /content/foo
!tar -xzf /content/drive/MyDrive/silent_speech/emg_data.tar.gz -C /content/silent_speech

In [ ]:
#!wget https://github.com/dgaddy/silent_speech_alignments/raw/main/text_alignments.tar.gz

In [6]:
!tar -xzf /content/drive/MyDrive/silent_speech/text_alignments.tar.gz -C /content/silent_speech

In [ ]:
%cd /content/silent_speech

# Environment Setup

In [13]:
!pip install pysndfile
#==1.0.28

In [14]:
!git clone https://github.com/NVIDIA/nv-wavenet.git nv_wavenet

Cloning into 'nv_wavenet'...
remote: Enumerating objects: 241, done.
remote: Total 241 (delta 0), reused 0 (delta 0), pack-reused 241
Receiving objects: 100% (241/241), 6.68 MiB | 16.21 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [30]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [29]:
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [31]:
%cd /content/silent_speech/nv_wavenet/pytorch
!make

/content/silent_speech/nv_wavenet/pytorch
nvcc -arch=sm_70 -std=c++11  --use_fast_math -lineinfo -maxrregcount 128 -I .. wavenet_infer.cu ../matrix.cpp -lz -Xcompiler -fPIC -shared -o libwavenet_infer.so
../nv_wavenet_conversions.cuh: In function ‘bool isDevicePtr(const void*)’:
../nv_wavenet_conversions.cuh:41:50: warning: ‘cudaPointerAttributes::memoryType’ is deprecated [-Wdeprecated-declarations]
     return (result == cudaSuccess) && (attributes.memoryType == cudaMemoryTypeDevice);
                                                  ^~~~~~~~~~
In file included from /usr/local/cuda/bin/../targets/x86_64-linux/include/builtin_types.h:59:0,
                 from /usr/local/cuda/bin/../targets/x86_64-linux/include/crt/host_runtime.h:58,
                 from /usr/lib/gcc/x86_64-linux-gnu/7/include/stddef.h:220:
/usr/local/cuda-10.1/targets/x86_64-linux/include/driver_types.h:1302:43: note: declared here
     __CUDA_DEPRECATED enum cudaMemoryType memoryType;
                             

In [32]:
!python build.py install

running install
running bdist_egg
running egg_info
writing nv_wavenet_ext.egg-info/PKG-INFO
writing dependency_links to nv_wavenet_ext.egg-info/dependency_links.txt
writing top-level names to nv_wavenet_ext.egg-info/top_level.txt
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:381: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'nv_wavenet_ext.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'nv_wavenet_ext' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-stron

In [22]:
!pip install absl-py librosa soundfile matplotlib scipy numba jiwer unidecode deepspeech==0.8.2 praat-textgrids

     |████████████████████████████████| 235 kB 8.6 MB/s 
     |████████████████████████████████| 8.3 MB 19.8 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149862 sha256=54041b1b96776a421f38bf39c005366f6851790f421e351d269059da4ba91106
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
  Created wheel for praat-textgrids: filename=praat_textgrids-1.3.1-py3-none-any.whl size=12397 sha256=04c3754507aee69e68aefcae43a746e874283a5b433a4edc8309c7fb78162f16
  Stored in directory: /root/.cache/pip/wheels/78/5e/7c/ebd0d2a251e76cc196edcb3781be75b68dfbf91f149060e6d5
Successfully built python-Levenshtein praat-textgrids


In [1]:
%cd /content/silent_speech

/content/silent_speech


In [35]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.pbmm
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.scorer

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   665  100   665    0     0   2021      0 --:--:-- --:--:-- --:--:--  2015
100  180M  100  180M    0     0  8758k      0  0:00:21  0:00:21 --:--:-- 5685k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   667  100   667    0     0   3473      0 --:--:-- --:--:-- --:--:--  3455
100  909M  100  909M    0     0  12.2M      0  0:01:14  0:01:14 --:--:-- 18.1M


# Running

In [34]:
!python wavenet_model.py --output_directory "./models/wavenet_model/" --silent_data_directories ""


['wavenet_model.py', '--output_directory', './models/wavenet_model/', '--silent_data_directories', '']
output example: (./emg_data/voiced_parallel_data/5-6, 194)
/content/silent_speech/nv_wavenet/pytorch/wavenet.py:46: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  self.conv.weight, gain=torch.nn.init.calculate_gain(w_init_gain))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
finished epoch 1 with error 2.18
 train error 2.36
saving model
Traceback (most recent call last):
  File "wavenet_model.py", line 163, in <module>
    train()
  File "wavenet_mo

In [ ]:

#!python transduction_model.py --data_size_fraction 0.5 --batch_size 1 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 2 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
!python transduction_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 32 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"



c993f9d7440ba2a5e30edc61a85a92ffade62a95

diff --git a/transduction_model.py b/transduction_model.py
index 24bc0d7..7848af3 100755
--- a/transduction_model.py
+++ b/transduction_model.py
@@ -17,13 +17,15 @@ from data_utils import phoneme_inventory, decollate_tensor
 
 from absl import flags
 FLAGS = flags.FLAGS
-flags.DEFINE_integer('model_size', 768, 'number of hidden dimensions')
+flags.DEFINE_integer('model_size', 384, 'number of hidden dimensions')
+#flags.DEFINE_integer('model_size', 768, 'number of hidden dimensions')
 flags.DEFINE_integer('num_layers', 6, 'number of layers')
 flags.DEFINE_integer('batch_size', 32, 'training batch size')
 flags.DEFINE_float('learning_rate', 1e-3, 'learning rate')
 flags.DEFINE_integer('learning_rate_patience', 5, 'learning rate decay patience')
 flags.DEFINE_integer('learning_rate_warmup', 500, 'steps of linear warmup')
 flags.DEFINE_string('start_training_from', None, 'start training from this model')
+#flags.DEFINE_float('data_size_fraction', 0

In [38]:
!python transduction_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"


c993f9d7440ba2a5e30edc61a85a92ffade62a95

diff --git a/transduction_model.py b/transduction_model.py
index 24bc0d7..c9789e5 100755
--- a/transduction_model.py
+++ b/transduction_model.py
@@ -208,7 +208,8 @@ def train_model(trainset, devset, device, save_sound_outputs=True, n_epochs=80):
         training_subset = trainset
     else:
         training_subset = torch.utils.data.Subset(trainset, list(range(int(len(trainset)*FLAGS.data_size_fraction))))
-    dataloader = torch.utils.data.DataLoader(training_subset, pin_memory=(device=='cuda'), collate_fn=devset.collate_fixed_length, num_workers=8, batch_sampler=SizeAwareSampler(trainset, 256000))
+    dataloader = torch.utils.data.DataLoader(training_subset, pin_memory=(device=='cuda'), collate_fn=devset.collate_fixed_length, num_workers=2, batch_sampler=SizeAwareSampler(trainset, 256000))
+#    dataloader = torch.utils.data.DataLoader(training_subset, pin_memory=(device=='cuda'), collate_fn=devset.collate_fixed_length, num_workers=8, batc

In [ ]:
!python evaluate.py --models ./models/transduction_model/model.pt --pretrained_wavenet_model ./models/wavenet_model/wavenet_model.pt --output_directory evaluation_output


In [ ]:
!python evaluate.py --models ./models/transduction_model/model.pt --pretrained_wavenet_model ./models/wavenet_model/wavenet_model.pt --output_directory evaluation_output --testset_file testset_origdev.json
